In [7]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm import tqdm
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize


In [5]:
annotated_post_df = pd.read_csv("/Users/brittanyharding/LHL-Projects/SM-suicide_detection/annotated_train_df/annotated_post_df.csv")
nonannotated_post_df = pd.read_csv("/Users/brittanyharding/LHL-Projects/SM-suicide_detection/nonannotated_test_df/nonannotated_post_df.csv")

### Create Slang Dictionary to Best Understand MH posts

In [9]:
slang_dict = {
    "mh": "mental health",
    "adhd": "attention deficit hyperactivity disorder",
    "asd": "autism spectrum disorder",
    "bpd": "borderline personality disorder",
    "ocd": "obsessive-compulsive disorder",
    "ptsd": "post-traumatic stress disorder",
    "gad": "generalized anxiety disorder",
    "mdd": "major depressive disorder",
    "sad": "social anxiety disorder or seasonal affective disorder",
    "bd": "bipolar disorder",
    "dx": "diagnosis",
    "tx": "treatment",
    "rx": "prescription",
    "cbt": "cognitive behavioral therapy",
    "dbt": "dialectical behavior therapy",
    "emdr": "eye movement desensitization and reprocessing",
    "act": "acceptance and commitment therapy",
    "ssri": "selective serotonin reuptake inhibitor",
    "snri": "serotonin and norepinephrine reuptake inhibitor",
    "nd": "neurodivergent",
    "nt": "neurotypical",
    "dsm": "diagnostic and statistical manual of mental disorders",
    "stims": "self-stimulatory behavior",
    "stimming": "self-stimulatory behavior",
    "tw": "trigger warning",
    "bp": "bipolar",
    "apd": "antisocial personality disorder",
    "avpd": "avoidant personality disorder",
    "did": "dissociative identity disorder",
    "spd": "schizoid personality disorder or sensory processing disorder",
    "ed": "eating disorder",
    "an": "anorexia nervosa",
    "bn": "bulimia nervosa",
    "bed": "binge eating disorder",
    "osfed": "other specified feeding or eating disorder",
    "sh": "self harm",
    "si": "suicidal ideation",
    "iop": "intensive outpatient",
    "ip": "inpatient",
    "tms": "transcranial magnetic stimulation",
    "ect": "electroconvulsive therapy",
    "np": "nurse practitioner",
    "pa": "physician's assistant",
    "psyd": "doctor of psychology",
    "md": "medical doctor",
    "lcsw": "licensed clinical social worker",
    "lmft": "licensed marriage and family therapist",
    "lpcc": "licensed professional clinical counselor",
    "erp": "exposure and response prevention",
    "nos": "not otherwise specified",
    "pe": "prolonged exposure",
    "sa": "social anxiety",
    "meds": "medications",
    "dpdr": "depersonalization-derealization",
    "pdoc": "psychiatrist",
    "tdoc": "therapist",
    "brain fog": "difficulty thinking clearly",
    "grounding": "techniques used to help stay in the present moment",
    "disso": "dissociation",
    "mania": "manic episode",
    "hypo": "hypomania",
    "zaps": "brain zaps",
    "med change": "changing medications",
    "titrate": "gradually increase or decrease medication dose",
    "psych ward": "psychiatric ward",
    "5150": "involuntary psychiatric hold",
    "in crisis": "experiencing a mental health emergency",
    "meltdown": "emotional breakdown",
    "shutdown": "a response to stress or overwhelm often experienced by people with autism",
    "spiraling": "quickly worsening mental health symptoms",
    "flashbacks": "intrusive memories of a traumatic event",
    "grounding techniques": "methods used to bring oneself back into the present moment",
    "gaslighting": "manipulative behavior to make someone doubt their own experiences",
    "ghosting": "ending a relationship by suddenly and without explanation withdrawing from all communication",
    "breadcrumbing": "the act of sending out flirtatious, but non-committal messages in order to lure a romantic partner",
    "shed": "self-harmed",
    "op": "original poster",
    "tl;dr": "too long; didn't read",
    "dae": "does anyone else",
    "iirc": "if i remember correctly",
    "ftfy": "fixed that for you",
    "ama": "ask me anything",
    "amaa": "ask me almost anything",
    "iama": "i am a...",
    "cmv": "change my view",
    "lpt": "life pro tip",
    "eli5": "explain like i'm 5",
    "imo": "in my opinion",
    "imho": "in my honest opinion",
    "ymmv": "your mileage may vary",
    "so": "significant other",
    "ysk": "you should know",
    "psa": "public service announcement",
    "til": "today i learned",
    'nsfl': "not safe for life",
    'nsfw': "not safe for work",
    'wip': "work in progress",
    'ikyky': "if you know you know",
    'finna': "getting ready to do something",
    'med': "medication",
    'meds': "medications",
    'od d': "overdosed",
    'od': "overdose",
    'sm': "social media",
    'pci': "post-crisis intervention",
    'hr': "human resources",
    "cap": "lie",
    "no cap": "no lie",
    "lit": "amazing or exciting",
    "on fleek": "perfect or flawless",
    "flex": "showing off or bragging",
    "clout": "influence or popularity",
    "savage": "fierce or ruthless",
    "GOAT": "greatest of all time",
    "bae": "before anyone else",
    "chill": "calm down",
    "thirsty": "desperate for attention",
    "woke": "aware or knowledgeable",
    "AF": "as fuck (emphasizing something)",
    "squad": "group of friends",
    "hater": "person who dislikes or criticizes others",
    "flexin'": "showing off or boasting",
    "gig": "job or event",
    "shook": "surprised or shocked",
    "vibe": "atmosphere or feeling",
    "thicc": "curvy or voluptuous",
    "salty": "bitter or resentful",
    "basic": "unoriginal or mainstream",
    "extra": "over the top or excessive",
    "depressy": "depressed",
    "grippy sock vacation": "psychiatric care",
    "doom scrolling": "obsessively scrolling",
    "sewerslide": "suicide",
    "menty b": "mental breakdown",
    "unalive": "die",
    "i had pasta tonight": "having suicidal thoughts",
    "i finished my shampoo and conditioner at the same time": "having suicidal thoughts",
    "plug": "drug dealer",
    "420": "marijuana",
    "burnout": "heavy drug user",
    "clucking": "withdrawal",
    "cold turkey": "abrupt withdrawal",
    "cooker": "heavy drug user",
    "dial-a-doping": "drug delivery",
    "doc": "drug of choice",
    "faded": "intoxicated",
    "🍭": "drug user or drug supplier",
    "molly": "mdma",
    "pnp": "party and play",
    "snowflake": "cocaine or overly sensitive person",
    "turnt": "under influence",
    "zombie": "heavy drug user",
    "kys": "kill yourself",
    "kms": "kill myself",
    "merked": "very drunk or beaten up",
    "#sue": "suicide",
    "dirl": "die in real life",
    "#ana": "anorexia",
    "#deb": "depression",
    "cat scratches": "superficial self-harm cuts",
    "styros": "self-harm cuts to dermis",
    "beans": "self-harm cuts to fat layer",
    "x": "ecstasy",
    "xan": "xanax",
    "back-to-school necklace": "noose reference",
    "#cuts": "self-harm discussion",
    "#cu46": "sexual meet-up",
    "#ednos": "unspecified eating disorder",
    "#kush": "marijuana discussion",
    "#mias": "bulimia discussion",
    "#secretsociety123": "self-harm community",
    "#selfharmmm": "self-harm discussion",
    "#svv": "self-harm discussion",
    "#tina": "crystal meth",
    "#thinsp": "unhealthy weight loss promotion",
    "#proana": "pro-anorexia promotion",
    "#promia": "pro-bulimia promotion",
    "yeet": "self-harm - cutting",
    "yeets": "self-harm wounds/scars (recent)",
    "yeeting": "cutting",
    "final yeet": "suicide attempt",
    "slicey bois": "razors",
    "barcode": "cluster of self-harm wounds/scars"
}


### Spell Check

In [10]:
# Create a spell checker object
spell_checker = SpellChecker()

# Add the slang terms to the spell checker's vocabulary
spell_checker.word_frequency.load_words(slang_dict.keys())

# Function to perform spell checking while preserving slang terms
def spell_check(text):
    # Tokenize the text into words
    words = text.split()
    corrected_words = []

    # Iterate over the words and check if they are in the spell checker's vocabulary
    for word in words:
        # Check if the word is a slang term
        if word.lower() in slang_dict:
            corrected_word = word
        else:
            # Check if the word is misspelled
            if word not in spell_checker:
                # Get the corrected spelling for the word
                corrected_word = spell_checker.correction(word)
                # Handle the case where the correction is None
                if corrected_word is None:
                    corrected_word = word
            else:
                corrected_word = word
        
        corrected_words.append(corrected_word)

    # Join the corrected words back into a string
    corrected_text = ' '.join(corrected_words) if corrected_words else ''

    return corrected_text


In [12]:
def preprocess_all(data_frame, column):
    # Create new column names for the preprocessed text
    new_column = 'preprocessed_' + column
    
    # Fill null values with empty strings
    data_frame[column].fillna('', inplace=True)
    
    # Remove punctuation
    data_frame[new_column] = data_frame[column].apply(lambda x: x.translate(str.maketrans("", "", string.punctuation)) if isinstance(x, str) else x)
    
    # Convert to lowercase
    data_frame[new_column] = data_frame[new_column].apply(lambda x: x.lower() if isinstance(x, str) else x)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    data_frame[new_column] = data_frame[new_column].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]) if isinstance(x, str) else x)
    
    # Stem the tokens
    stemmer = PorterStemmer()
    data_frame[new_column] = data_frame[new_column].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]) if isinstance(x, str) else x)
    
    return data_frame

In [13]:
import string

def basic_clean(data_frame, column):
    # Create new column names for the preprocessed text
    new_column = 'lowercase_punc_removal_' + column
    
    # Remove punctuation and convert to lowercase
    data_frame[new_column] = data_frame[column].apply(lambda x: x.translate(str.maketrans("", "", string.punctuation)).lower() if isinstance(x, str) else x)
    
    return data_frame


In [14]:
def tokenize_text(data_frame, column):
    # Create new column name for the tokenized text
    tokenized_column = 'tokenized_' + column
    
    # Fill null values with empty strings
    data_frame[column].fillna('', inplace=True)
    
    # Tokenize the text
    data_frame[tokenized_column] = data_frame[column].apply(lambda x: word_tokenize(x) if isinstance(x, str) else x)
    
    return data_frame

## Clean Annotated Data

In [11]:
# Apply the spell_check function to the 'post' column in your annotated_post_df DataFrame
annotated_post_df['post'] = annotated_post_df['post'].apply(spell_check)

# Apply the preprocess_all function to annotated_post_df
annotated_post_df = preprocess_all(annotated_post_df, 'post')

# Apply the basic_clean function to annotated_post_df
annotated_post_df = basic_clean(annotated_post_df, 'post')

# Apply the tokenize_text function to annotated_post_df
annotated_post_df = tokenize_text(annotated_post_df, 'preprocessed_post')
annotated_post_df = tokenize_text(annotated_post_df, 'lowercase_punc_removal_post')

In [21]:
from textblob import TextBlob

annotated_post_df['sentiment_polarity'] = annotated_post_df['post'].apply(lambda x: TextBlob(x).sentiment.polarity)
annotated_post_df['sentiment_subjectivity'] = annotated_post_df['post'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [23]:
# Save annotated_post_df to clean_annotated_posts.csv
annotated_post_df.to_csv('clean_annotated_posts.csv', index=False)


## Clean Non-Annotated Data

In [17]:
# # Apply the spell_check function to the 'post' column in your nonannotated_post_df DataFrame
# nonannotated_post_df['post'] = nonannotated_post_df['post'].apply(spell_check)

# # Apply the preprocess_all function to annotated_post_df
# nonannotated_post_df = preprocess_all(nonannotated_post_df, 'post')

# # Apply the basic_clean function to annotated_post_df
# nonannotated_post_df = basic_clean(nonannotated_post_df, 'post')

# Apply the tokenize_text function to annotated_post_df
nonannotated_post_df = tokenize_text(nonannotated_post_df, 'preprocessed_post')
nonannotated_post_df = tokenize_text(nonannotated_post_df, 'lowercase_punc_removal_post')

In [18]:
# Save annotated_post_df to clean_annotated_posts.csv
nonannotated_post_df.to_csv('clean_non_annotated_posts.csv', index=False)
